In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import re
import xgboost
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

Training a regressor to predict housing prices in a city of northeast Brazil with data scrapped from OLX listings

**Results**: much of the data is dirty because of site users inserting wrong/incomplete information. Only features that helped predict prices consistently were meter area and location. Would be nice if the data were more real and contained geolocalization info. 

In [ ]:
tf.random.set_seed(33)
np.random.seed(33)

In [ ]:
data = r'/content/drive/MyDrive/Colab Notebooks/datafiles/jp_data_07_22.csv'

In [ ]:
df = pd.read_csv(data)
df = df.drop("Unnamed: 0", axis=1)
df

,area,bedrooms,location,price
0,277m²,4 quartos,"João Pessoa, Manaíra",R$ 1.300.000
1,7300m²,3 quartos,"João Pessoa, Gramame",R$ 230.000
2,277m²,4 quartos,"João Pessoa, Manaíra",R$ 1.300.000
3,13000m²,3 quartos,"João Pessoa, Cristo Redentor",R$ 235.000
4,NaN,NaN,NaN,NaN
...,...,...,...,...
5695,300m²,3 quartos,"João Pessoa, Pedro Gondim",R$ 400.000
5696,235m²,3 quartos,"João Pessoa, Portal do Sol",R$ 570.000
5697,180m²,2 quartos,"João Pessoa, Expedicionários",R$ 560.000
5698,58m²,2 quartos,"João Pessoa, Funcionários",R$ 155.000


In [ ]:
original = pd.read_csv(data).drop("Unnamed: 0", axis=1)

In [ ]:
def fmt_area(area):
  if isinstance(area, str) and "m²" in area:
    area = area.replace("m²", '')
    return float(area)
  else:
    return np.nan

def fix_area(area):
  if isinstance(area, float):
    if area >= 2000 and area <= 100000:
      area =  area / 100
    elif area > 100000 and area < 1000000:
      area =  area / 1000
    elif area > 1000000:
      area = np.nan
    
    if area < 50:
      area = np.nan
    else:
      return float(area)

def fmt_bedrooms(bedrooms):
  if not "quarto" in str(bedrooms):
    return np.nan
  match = re.search("\d", bedrooms)
  if match is not None:
    return int(match.group(0))
  else:
    return np.nan

def fmt_values(num):
  if not isinstance(num, str):
    return num
  else:
    price = re.sub(r"R[^0-9] ",'', num)
    price = re.sub(r"\.",'', price)
    price = float(price)
  
  if price < 50000 or price > 8000000:
    return np.nan

  return price



In [ ]:
df.area = df.area.apply(fmt_area)
df.area = df.area.apply(fix_area)
df.bedrooms = df.bedrooms.apply(fmt_bedrooms)

In [ ]:
df.price = df.price.apply(fmt_values)

In [ ]:
original.iloc[72]

area                             1300m²
bedrooms              5 ou mais quartos
location    João Pessoa, Jardim Oceania
price                      R$ 4.500.000
Name: 72, dtype: object

In [ ]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4067 entries, 0 to 5699
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   area      4067 non-null   float64
 1   bedrooms  4067 non-null   float64
 2   location  4067 non-null   object 
 3   price     4067 non-null   float64
dtypes: float64(3), object(1)
memory usage: 158.9+ KB


In [ ]:
locations = df.location.unique()
locations

array(['João Pessoa, Manaíra', 'João Pessoa, Gramame',
       'João Pessoa, Cristo Redentor', 'João Pessoa, Varjão',
       'João Pessoa, José Américo de Almeida',
       'João Pessoa, Distrito Industrial', 'João Pessoa, Ernani Sátiro',
       'João Pessoa, Paratibe', 'João Pessoa, Altiplano Cabo Branco',
       'João Pessoa, Valentina de Figueiredo',
       'João Pessoa, Funcionários', 'João Pessoa, Mangabeira',
       'João Pessoa, Aeroclube', 'João Pessoa, Indústrias',
       'João Pessoa, Brisamar', 'João Pessoa, Oitizeiro',
       'João Pessoa, Portal do Sol', 'João Pessoa, Jaguaribe',
       'João Pessoa, Bessa', 'João Pessoa, Ernesto Geisel',
       'João Pessoa, Estados', 'João Pessoa, Bancários',
       'João Pessoa, Jardim São Paulo', 'João Pessoa, Jardim Oceania',
       'João Pessoa, Expedicionários', 'João Pessoa, João Agripino',
       'João Pessoa, Ipês', 'João Pessoa, Mumbaba', 'João Pessoa, Cuiá',
       'João Pessoa, Muçumagro', 'João Pessoa, Tambiá',
       'Cabedelo

In [ ]:
df["meter_price"] =  df.apply(lambda row: round(row.price / row.area, 2), axis=1)

In [ ]:
df[df.location == "João Pessoa, Cabo Branco"].meter_price.mean()

2355.98641509434

Removing outliers based on price per square meter

In [ ]:
stats = {}
for loc in locations:
  dff = df[df.location == loc]
  stats[loc] = {"mean": dff.meter_price.mean(), "std": dff.meter_price.std()}


In [ ]:
def get_zscore(row):
  stat = stats[row.location]
  mean = stat["mean"]
  std = stat["std"]
  return (row.meter_price - mean )  / std

df["z_score"] = df.apply( get_zscore, axis =1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [ ]:
dff =  df[(df.z_score >= - 3 ) & (df.z_score <= 3 ) ]           # syntax #df[(df['col1'] >= 1) & (df['col1'] <=1 )]

In [ ]:
dff

,area,bedrooms,location,price,meter_price,z_score
0,277.0,4.0,"João Pessoa, Manaíra",1300000.0,4693.14,0.299565
1,73.0,3.0,"João Pessoa, Gramame",230000.0,3150.68,0.477407
2,277.0,4.0,"João Pessoa, Manaíra",1300000.0,4693.14,0.299565
3,130.0,3.0,"João Pessoa, Cristo Redentor",235000.0,1807.69,-0.496481
6,250.0,3.0,"João Pessoa, Manaíra",1300000.0,5200.00,0.568619
...,...,...,...,...,...,...
5695,300.0,3.0,"João Pessoa, Pedro Gondim",400000.0,1333.33,-0.617782
5696,235.0,3.0,"João Pessoa, Portal do Sol",570000.0,2425.53,-1.587437
5697,180.0,2.0,"João Pessoa, Expedicionários",560000.0,3111.11,0.311313
5698,58.0,2.0,"João Pessoa, Funcionários",155000.0,2672.41,1.256131


In [ ]:
def standardize_features( dataframe: pd.DataFrame, feature_names: list, operation="standardize"):
  """standardizes numerical features and returns a dataframe"""
  from sklearn.preprocessing import StandardScaler, Normalizer
  if operation == "normalize":
    operator = Normalizer()
  else:
    operator = StandardScaler()
  df = dataframe
  for column_name in feature_names:
      raw_data = np.array(df[column_name]).reshape(-1, 1)
      transformed = operator.fit_transform(raw_data)
      ready = pd.DataFrame(transformed, columns=[column_name])
      df = df.drop(column_name, axis=1).join(ready)
  return df

In [ ]:
X = dff.drop(["price", "meter_price", "z_score","bedrooms"], axis=1,)

#One hot encoding  
X = pd.get_dummies(X, prefix='', prefix_sep='', columns=['location'],drop_first=True)
Y = dff.price

In [ ]:
X.shape[1]

68

In [ ]:
x_train_full, x_test, y_train_full, y_test =  train_test_split(X, Y, test_size=0.1)
x_valid, x_train =  x_train_full[:300], x_train_full[300:]
y_valid, y_train =  y_train_full[:300], y_train_full[300:]

In [ ]:
x_train.shape

(3311, 68)

In [ ]:
forests = RandomForestRegressor(n_estimators=100, random_state=32)
xgb = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.2, random_state=42, max_depth=10)
selected_model = xgb

selected_model.fit(x_train, y_train)
pred = selected_model.predict(x_test)
squared_error = mean_squared_error(pred, y_test, squared=False)
absolute_error = mean_absolute_error(pred, y_test)
print(absolute_error, squared_error)

[20:36:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
180905.6829329913 465106.8946409851


In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True, monitor="loss")



sample_to_normalize = x_train
norm_layer = keras.layers.Normalization(axis=1)
norm_layer.adapt(tf.constant(sample_to_normalize))

neural = keras.models.Sequential([
norm_layer,
keras.layers.Dense(63, activation="selu", kernel_initializer="lecun_normal", input_shape=X.shape[1:]),
keras.layers.Dense(63, kernel_initializer="lecun_normal", activation="selu"),
keras.layers.Dense(1, activation=None),
])

optimizer = keras.optimizers.SGD(learning_rate=0.0005, nesterov=True) #keras.optimizers.RMSprop(lr=0.001) 

neural.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer= optimizer)

history = neural.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=1000, callbacks=[early_stopping_cb])

Epoch 1/1000
104/104 [==============================] - 2s 7ms/step - loss: 659763.1875 - val_loss: 560762.1875
Epoch 2/1000
 46/104 [============>.................] - ETA: 0s - loss: 676072.5625

KeyboardInterrupt: ignored

In [ ]:
sample = x_test[10:20]
sample_answers = y_test[10:20]

In [ ]:
neural.predict(sample)

In [ ]:
list(sample_answers)

In [ ]:
pred = neural.predict(x_test)
real = y_test

squared_error = mean_squared_error(pred, real, squared=False)
absolute_error = mean_absolute_error(pred, real)
print(absolute_error, squared_error)

In [ ]:
locations

In [ ]:
def predict_new(model, X, area, location, bedrooms=None):
  locations = list(X.columns)
  size=X.shape[1]
  entry = np.zeros(size)
  try:
    locations = list(locations)
    location_idx = locations.index(location)
    entry[location_idx] = 1
  except ValueError:
    pass
  entry[0] = area
  #entry[1] = bedrooms
  entry = entry.reshape(1,size)
  return model.predict(entry)

In [ ]:
predict_new(neural, X, 182. ,"João Pessoa, José Américo de Almeida")